In [1]:
import pandas as pd

pd.set_option("display.max_columns",None)


In [2]:
dtypes={"BankZip":str, "BusinessAge":str}
foiaData=pd.read_csv("foia-7afy2010-fy2019-asof-220930.csv", encoding="ISO-8859-1", dtype=dtypes)

In [3]:
# Drop rows where BankZip is M5H 1 or BankName is MISSINGMAINBANKID
bankzip_condition = foiaData["BankZip"] == "M5H 1"
bankname_condition = foiaData["BankName"] == "MISSINGMAINBANKID"
drop_condition = bankzip_condition | bankname_condition
foiaData.drop(foiaData[drop_condition].index, inplace=True)

In [4]:
# Drop rows with missing Borrower Name
foiaData.dropna(subset=["BorrName"],inplace=True)

In [5]:
# Drop the BankFDICNumber and BankNCUANumber columns
columns_to_drop=["BankFDICNumber","BankNCUANumber"]
foiaData.drop(columns_to_drop,axis=1,inplace=True)

In [6]:
# Create a HasFranchise column based on FranchiseNamecolumn
foiaData["HasFranchise"]=foiaData["FranchiseCode"].notna().astype(int)

In [7]:
# Drop the FranchiseCode and FranchiseName columns
columns_to_drop=["FranchiseCode","FranchiseName"]
foiaData.drop(columns_to_drop,axis=1,inplace=True)

In [8]:
# Fill missing values in NaicsCode, NaicsDescription,
# CongressionalDistrict, and BusinessType with the most frequent value
for column in ["NaicsCode", "NaicsDescription",
               "CongressionalDistrict", "BusinessType"]:
    most_frequent_value = foiaData[column].mode()[0]
    foiaData[column].fillna(most_frequent_value, inplace=True)

In [9]:
# Fill missing values in ProjectCounty with the most frequent value in
# ProjectState == CT
ctmode_projectstate=foiaData.loc[foiaData["ProjectState"]=="CT","ProjectCounty"].mode()[0]
foiaData["ProjectCounty"].fillna(ctmode_projectstate,inplace=True)

In [10]:
# Drop the BusinessAge column
foiaData.drop("BusinessAge",axis=1,inplace=True)

In [11]:
foiaData.head()

,AsOfDate,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,subpgmdesc,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,HasFranchise
0,20220930,7A,RIVERLAND LASER LLC,703 ST CROIX STREET,RIVER FALLS,WI,54022,First National Community Bank,109 E 2nd St,NEW RICHMOND,WI,54017,491000,441900.0,10/01/2009,2010,10/01/2009,OTH 7A,Guaranty,6.00,120,332812.0,"Metal Coating, Engraving (except Jewelry and S...",PIERCE,WI,WISCONSIN DISTRICT OFFICE,7.0,CORPORATION,PIF,08/31/2016,NaN,0,0,11,0
1,20220930,7A,MARTINEZ AND MARTINEZ INC.,2430 DALY AVE,LOS ANGELES,CA,90031,Gain FCU,1800 W Magnolia Blvd,BURBANK,CA,91506,35000,31500.0,10/01/2009,2010,04/01/2011,OTH 7A,Guaranty,6.50,60,541213.0,Tax Preparation Services,LOS ANGELES,CA,LOS ANGELES DISTRICT OFFICE,28.0,CORPORATION,PIF,04/30/2016,NaN,0,1,9,0
2,20220930,7A,Erez & Koby Genesis Investment,"10999 Reed Hartman Hwy, SUit",CINCINNATI,OH,45242,"JPMorgan Chase Bank, National Association",1111 Polaris Pkwy,COLUMBUS,OH,43240,100500,50250.0,10/01/2009,2010,10/01/2009,SBA EXPRES,FA$TRK (Small Loan Express),4.51,54,722110.0,Full-Service Restaurants,HAMILTON,OH,COLUMBUS DISTRICT OFFICE,1.0,CORPORATION,PIF,05/31/2012,NaN,0,0,20,0
3,20220930,7A,Square Burger LLC,115 Kentucky Street,McKinney,TX,75069,American Bank of Commerce,530 E Hwy 62/82,WOLFFORTH,TX,79382,288000,259200.0,10/01/2009,2010,10/01/2009,PLP,Guaranty,6.00,120,722110.0,Full-Service Restaurants,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,3.0,CORPORATION,PIF,07/31/2014,NaN,0,0,18,0
4,20220930,7A,HIT FITNESS LLC,614 A SOUTH MAIN STREET,MAULDIN,SC,29662,Truist Bank,214 N Tryon St,CHARLOTTE,NC,28202,37000,18500.0,10/01/2009,2010,10/01/2009,SBA EXPRES,FA$TRK (Small Loan Express),7.50,60,812199.0,Other Personal Care Services,GREENVILLE,SC,SOUTH CAROLINA DISTRICT OFFICE,4.0,CORPORATION,PIF,09/30/2013,NaN,0,0,6,0


In [12]:
#Convert the "BankZip" column to type int
foiaData["BankZip"]=foiaData["BankZip"].astype(int)

# Convert the first column "AsOfDate" to a date
foiaData["AsOfDate"]=pd.to_datetime(foiaData["AsOfDate"], format="%Y%m%d")

#Convert other columns to datetime format
for column_to_date in ["ApprovalDate", "FirstDisbursementDate",
               "PaidInFullDate", "ChargeOffDate"]:
    foiaData[column_to_date]=pd.to_datetime(foiaData[column_to_date])



In [13]:
foiaData.dtypes

AsOfDate                 datetime64[ns]
Program                          object
BorrName                         object
BorrStreet                       object
BorrCity                         object
BorrState                        object
BorrZip                           int64
BankName                         object
BankStreet                       object
BankCity                         object
BankState                        object
BankZip                           int32
GrossApproval                     int64
SBAGuaranteedApproval           float64
ApprovalDate             datetime64[ns]
ApprovalFiscalYear                int64
FirstDisbursementDate    datetime64[ns]
DeliveryMethod                   object
subpgmdesc                       object
InitialInterestRate             float64
TermInMonths                      int64
NaicsCode                       float64
NaicsDescription                 object
ProjectCounty                    object
ProjectState                     object


In [14]:
foiaData.head(5)

,AsOfDate,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,subpgmdesc,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,HasFranchise
0,2022-09-30,7A,RIVERLAND LASER LLC,703 ST CROIX STREET,RIVER FALLS,WI,54022,First National Community Bank,109 E 2nd St,NEW RICHMOND,WI,54017,491000,441900.0,2009-10-01,2010,2009-10-01,OTH 7A,Guaranty,6.00,120,332812.0,"Metal Coating, Engraving (except Jewelry and S...",PIERCE,WI,WISCONSIN DISTRICT OFFICE,7.0,CORPORATION,PIF,2016-08-31,NaT,0,0,11,0
1,2022-09-30,7A,MARTINEZ AND MARTINEZ INC.,2430 DALY AVE,LOS ANGELES,CA,90031,Gain FCU,1800 W Magnolia Blvd,BURBANK,CA,91506,35000,31500.0,2009-10-01,2010,2011-04-01,OTH 7A,Guaranty,6.50,60,541213.0,Tax Preparation Services,LOS ANGELES,CA,LOS ANGELES DISTRICT OFFICE,28.0,CORPORATION,PIF,2016-04-30,NaT,0,1,9,0
2,2022-09-30,7A,Erez & Koby Genesis Investment,"10999 Reed Hartman Hwy, SUit",CINCINNATI,OH,45242,"JPMorgan Chase Bank, National Association",1111 Polaris Pkwy,COLUMBUS,OH,43240,100500,50250.0,2009-10-01,2010,2009-10-01,SBA EXPRES,FA$TRK (Small Loan Express),4.51,54,722110.0,Full-Service Restaurants,HAMILTON,OH,COLUMBUS DISTRICT OFFICE,1.0,CORPORATION,PIF,2012-05-31,NaT,0,0,20,0
3,2022-09-30,7A,Square Burger LLC,115 Kentucky Street,McKinney,TX,75069,American Bank of Commerce,530 E Hwy 62/82,WOLFFORTH,TX,79382,288000,259200.0,2009-10-01,2010,2009-10-01,PLP,Guaranty,6.00,120,722110.0,Full-Service Restaurants,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,3.0,CORPORATION,PIF,2014-07-31,NaT,0,0,18,0
4,2022-09-30,7A,HIT FITNESS LLC,614 A SOUTH MAIN STREET,MAULDIN,SC,29662,Truist Bank,214 N Tryon St,CHARLOTTE,NC,28202,37000,18500.0,2009-10-01,2010,2009-10-01,SBA EXPRES,FA$TRK (Small Loan Express),7.50,60,812199.0,Other Personal Care Services,GREENVILLE,SC,SOUTH CAROLINA DISTRICT OFFICE,4.0,CORPORATION,PIF,2013-09-30,NaT,0,0,6,0


In [15]:
foiaData.to_csv("foiaData.csv",index=False)

In [16]:
foiaData.tail()

,AsOfDate,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,BankName,BankStreet,BankCity,BankState,BankZip,GrossApproval,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,subpgmdesc,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,HasFranchise
545746,2022-09-30,7A,AML Transport LLC,7641 Division Ave S,Grand Rapids,MI,49548,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215,16500,8250.0,2019-09-30,2019,2019-10-31,SBA EXPRES,FA$TRK (Small Loan Express),5.60,60,484110.0,"General Freight Trucking, Local",KENT,MI,MICHIGAN DISTRICT OFFICE,2.0,CORPORATION,PIF,2021-04-30,NaT,0,0,2,0
545747,2022-09-30,7A,Super Swim Schools LLC,7000 E. Mayo Blvd.,Phoenix,AZ,85054,KeyBank National Association,127 Public Sq,CLEVELAND,OH,44114,1782500,1336875.0,2019-09-30,2019,2020-08-31,PLP,Guaranty,7.25,120,611620.0,Sports and Recreation Instruction,MARICOPA,AZ,ARIZONA DISTRICT OFFICE,6.0,CORPORATION,EXEMPT,NaT,NaT,0,0,60,1
545748,2022-09-30,7A,MASSAPEQUA 143 LLC,5129 Merrick Road,Massapequa Park,NY,11762,"TD Bank, National Association",2035 Limestone Rd,WILMINGTON,DE,19808,25000,12500.0,2019-09-30,2019,2019-12-31,SBA EXPRES,FA$TRK (Small Loan Express),10.74,120,713940.0,Fitness and Recreational Sports Centers,NASSAU,NY,MELVILLE BRANCH OFFICE,2.0,CORPORATION,EXEMPT,NaT,NaT,0,1,0,1
545749,2022-09-30,7A,Sportspedic LLC,25826 JANES CT,CHANTILLY,VA,20152,"Wells Fargo Bank, National Association",101 N Philips Ave,SIOUX FALLS,SD,57104,5000,2500.0,2019-09-30,2019,NaT,SBA EXPRES,FA$TRK (Small Loan Express),10.75,120,423450.0,"Medical, Dental, and Hospital Equipment and Su...",LOUDOUN,VA,WASHINGTON DISTRICT OFFICE,10.0,CORPORATION,COMMIT,NaT,NaT,0,1,0,0
545750,2022-09-30,7A,Tulum Homes LLC,2814 SPRINGVIEW LN,MESQUITE,TX,75181,"Wells Fargo Bank, National Association",101 N Philips Ave,SIOUX FALLS,SD,57104,10000,5000.0,2019-09-30,2019,2019-10-17,SBA EXPRES,FA$TRK (Small Loan Express),11.75,120,236118.0,Residential Remodelers,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,5.0,CORPORATION,EXEMPT,NaT,NaT,0,1,0,0
